In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Year Availability Metdata
- traffic counter entries in aadf dataset have differing survey year availability
- this metadata is needed to ensure survey years span those in the accident dataset
- this notebook make a dataframe of the aadf traffic counter survey year metadata
- this metadata will be merged into the "df_road" dataframe built by BuildRoadDataframe_v3p1.ipynb
- the user must have imported dft_traffic_counts_aadf.csv to run this notebook (the resulting dataframe is, however, included in the repository)

In [2]:
######THE USER MUST IMPORT THE BELOW FILE THEMSELVES, SEE README###########
path = 'dft_traffic_counts_aadf.csv'
df = pd.read_csv(path,low_memory=False)

In [9]:
IDs = np.unique(df.Count_point_id.values)
Nid = len(IDs)

YearAvail = [None]*Nid
NYear = np.zeros(Nid).astype(int)
update = 20e3
for ii,ID in enumerate(IDs):
    I = df.Count_point_id == ID
    YearAvail[ii] = np.sort(np.unique(df.loc[I,'Year'].values))
    NYear[ii] = len(YearAvail[ii])
    if (ii+1)%update == 0:
        print('%i of % done'%(ii+1,Nid))

20000 of  45576one
40000 of  45576one


In [10]:
#detect if there are any gaps in coverage
gaps = np.ones(Nid).astype(int)
for ii in range(Nid):
    if NYear[ii] > 1:
        gaps[ii] = np.diff(YearAvail[ii]).max()        

In [ ]:
#extract the start year and end year
YearStart = np.zeros(Nid).astype(int)
YearEnd = np.zeros(Nid).astype(int)
for ii in range(Nid):
    YearStart[ii] = YearAvail[ii][0]
    YearEnd[ii] = YearAvail[ii][-1]

#### Sample of metadata DataFrame

In [6]:
df.sample(5)

,Count_point_id,N_year,Year_start,Year_end,Max_year_gap
6547,36082,22,2000,2021,1
25722,806371,3,2019,2021,1
43943,966072,10,2000,2009,1
24941,805446,2,2018,2019,1
28618,809610,1,2019,2019,1


#### DataFrame can be saved for later use

In [16]:
save_path = 'Dataframes/aadf_year_meta.csv'
df = pd.DataFrame({'Count_point_id':IDs,'N_year':NYear,'Year_start':YearStart,'Year_end':YearEnd,'Max_year_gap':gaps})
df.to_csv(save_path,index_label=False)